In [1]:
import tools.log_parser as log_parser
import tools.backtester as backtester
import sys
import datamodel
import collections

In [9]:
LOG_PATH = "trader4_log.log"

LIMITS = backtester.KNOWN_LIMITS

In [10]:
log = log_parser.parse_log(LOG_PATH, parse_trader_log_as_object=True)
sandbox_logs = log.sandbox_logs

In [14]:
order_books = collections.defaultdict(lambda: [])
for entry in sandbox_logs:
    trader_log: datamodel.TradingState = entry.trader_log
    trade_order_depths = trader_log.order_depths
    for k, v in trade_order_depths.items():
        order_books[k].append((trader_log.timestamp, v))


In [15]:
order_books.keys()

dict_keys(['AMETHYSTS', 'STARFRUIT'])

In [17]:
def update_dp(dp, price, qty):
    assert qty in (-1, 1)
    new_dp = [None] * len(dp)
    for i in range(len(dp)):
        new_dp[i] = dp[i]
        if 0 <= i + qty < len(dp):
            new_dp[i] = max(new_dp[i], dp[i+qty] + (qty * price))
    for i in range(len(dp)):
        dp[i] = new_dp[i]


In [18]:
def sgn(x):
    return int(x > 0) - int(x < 0)

print(sgn(1), sgn(-1))

1 -1


In [32]:
for symbol, trades in order_books.items():
    print(symbol)
    dp = [-float("inf")] * (LIMITS[symbol] * 2 + 1)
    dp[LIMITS[symbol]] = 0
    for t, v in trades:
        for book in (v.buy_orders, v.sell_orders):
            for price, qty in book.items():
                price = int(price)
                qty = int(qty)
                for _ in range(abs(qty)):
                    update_dp(dp, price, sgn(qty))
    print(dp[LIMITS[symbol]], dp)

AMETHYSTS
1448 [201438, 191440, 181442, 171444, 161446, 151448, 141448, 131448, 121448, 111448, 101448, 91448, 81448, 71448, 61448, 51448, 41448, 31448, 21448, 11448, 1448, -8552, -18552, -28552, -38552, -48552, -58552, -68552, -78552, -88552, -98552, -108552, -118552, -128552, -138552, -148552, -158552, -168552, -178552, -188554, -198556]
STARFRUIT
2230 [101273, 96323, 91373, 86423, 81473, 76523, 71573, 66623, 61671, 56719, 51767, 46814, 41861, 36908, 31954, 27000, 22046, 17092, 12138, 7184, 2230, -2724, -7678, -12632, -17586, -22540, -27494, -32448, -37402, -42356, -47310, -52264, -57218, -62172, -67126, -72080, -77034, -81988, -86942, -91896, -96850]
